<a href="https://colab.research.google.com/github/sunnysavita10/Generative-AI-Indepth-Basic-to-Advance/blob/main/MergerRetriever_and_LongContextReorder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the Require Libraries

In [7]:
!pip install -qU langchain chromadb huggingface_hub sentence-transformers pypdf openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 23.3 M

In [8]:
!pip -q install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.2 MB/s eta 0:00:00


# Let's Load the Data Now...

In [9]:
from langchain.document_loaders import PyPDFLoader

In [10]:
# !mkdir model
# !wget -O /content/model/zephyr-7b-beta.Q4_0.gguf  https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_0.gguf?download=true

In [11]:
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
loader_rag  = PyPDFLoader("/content/Retrieval-Augmented-Generation-for-NLP.pdf")

In [14]:
documnet_rag = loader_rag.load()

In [15]:
print(len(documnet_rag))

19


In [16]:
loader_transf = PyPDFLoader("/content/Transformer.pdf")

In [17]:
documnet_transf = loader_transf.load()

In [18]:
print(len(documnet_transf))


15


# Let's create the text splitter for chunking

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)

In [21]:
text_rag = text_splitter.split_documents(documnet_rag)

In [22]:
text_transf = text_splitter.split_documents(documnet_transf)

In [23]:
print(len(text_rag))

184


In [24]:
print(len(text_transf))

103


# Load the Embedding Model to Convert the Data into Vector

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings,HuggingFaceBgeEmbeddings

In [27]:
# hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [28]:
hf_bge_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# from google.colab import userdata
# OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
# import os
# os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [ ]:
# openai_embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


# Now ingest the Data into the Chroma Database

In [29]:
from langchain.vectorstores import Chroma
import chromadb

In [30]:
import os
os.getcwd()

'/content'

In [31]:
CURRENT_DIR = os.path.dirname(os.path.abspath("."))

In [32]:
CURRENT_DIR

'/'

In [33]:
DB_DIR = os.path.join(CURRENT_DIR, "/content/db")

In [34]:
DB_DIR

'/content/db'

In [35]:
client_settings = chromadb.config.Settings(
    is_persistent=True,
    persist_directory=DB_DIR,
    anonymized_telemetry=False,
)

In [36]:
rag_vectorstore = Chroma.from_documents(text_rag,
                                       hf_bge_embeddings,
                                       client_settings=client_settings,
                                       collection_name="rag",
                                       collection_metadata={"hnsw":"cosine"},
                                       persist_directory="/store/rag")

In [37]:
transf_vectorstore = Chroma.from_documents(text_transf,
                                       hf_bge_embeddings,
                                       client_settings=client_settings,
                                       collection_name="transf",
                                       collection_metadata={"hnsw":"cosine"},
                                       persist_directory="/store/transf")

 # Now Crearte a Retriever

In [38]:
retriever_rag = rag_vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True})

In [39]:
retriever_transf = transf_vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True})

# Let's Merge both Retriever

# It is also called lord of retriever(LOTR)

In [40]:
from langchain.retrievers.merger_retriever import MergerRetriever

In [41]:
lotr = MergerRetriever(retrievers=[retriever_rag, retriever_transf])

In [42]:
for chunks in lotr.get_relevant_documents("What is transformer architecture"):
    print(chunks.page_content)
    print("*"*20)

work. These successes leverage different retrieval-based architectures and optimization techniques to
achieve strong performance on a single task, while we show that a single retrieval-based architecture
can be ﬁne-tuned for strong performance on a variety of tasks.
Memory-based Architectures Our document index can be seen as a large external memory for
neural networks to attend to, analogous to memory networks [ 64,55]. Concurrent work [ 14] learns
********************
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
********************
athttps://huggingfac

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [43]:
for chunks in lotr.get_relevant_documents("What is rag token?"):
    print(chunks.page_content)
    print("*"*30)

index) ﬁxed, only ﬁne-tuning the query encoder BERT qand the BART generator.
2.5 Decoding
At test time, RAG-Sequence and RAG-Token require different ways to approximate arg maxyp(y|x).
RAG-Token The RAG-Token model can be seen as a standard, autoregressive seq2seq genera-
tor with transition probability: p′
θ(yi|x,y 1:i−1) =∑
z∈top-k(p(·|x))pη(zi|x)pθ(yi|x,zi,y1:i−1)To
decode, we can plug p′
θ(yi|x,y 1:i−1)into a standard beam decoder.
******************************
Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the
English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.
ModelBLEU Training Cost (FLOPs)
EN-DE EN-FR EN-DE EN-FR
ByteNet [18] 23.75
Deep-Att + PosUnk [39] 39.2 1.0·1020
GNMT + RL [38] 24.6 39.92 2.3·10191.4·1020
ConvS2S [9] 25.16 40.46 9.6·10181.5·1020
MoE [32] 26.03 40.56 2.0·10191.2·1020
Deep-Att + PosUnk Ensemble [39] 40.4 8.0·1020
******************************
Doc 5Figure 2: RAG-Token 

## See this result is too much messy now lets refine it according to the question and overcome the situation of lost in middle

# Now After understanding step by step it create a pipeline for LLM

In [44]:
from langchain.document_transformers import (
    EmbeddingsClusteringFilter,
    EmbeddingsRedundantFilter,
)
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_transformers import LongContextReorder

In [45]:
from re import search
filter = EmbeddingsRedundantFilter(embeddings=hf_bge_embeddings)
reordering = LongContextReorder()
pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
compression_retriever_reordered = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr,search_kwargs={"k": 3, "include_metadata": True}
)


In [46]:
"""docs = compression_retriever_reordered.get_relevant_documents("What is esops?")
print(len(docs))
#
print(docs[0].page_content)"""

'docs = compression_retriever_reordered.get_relevant_documents("What is esops?")\nprint(len(docs))\n#\nprint(docs[0].page_content)'

# Load the model from huggingface

In [55]:
!pip -q install llama-cpp-python
!pip -q install bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 12.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.85-cp310-cp310-linux_x86_64.whl size=2873144 sha256=087a167c8c7c46263cfa5c351ad282ecbb07d064b63782ca60053a79b896fdd3
  Stored in directory: /root/.cache/pip/wheels/3f/e8/4e/29a754f9175ef52b6481cd75e3af4de38bf6dfa9c2972f75d4
Successfully built llama-cpp-python


In [48]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline

In [49]:
# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    model_name: Name or path of the model to be loaded.
    return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

In [50]:
# initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    model_name: Name or path of the model for tokenizer initialization.
    return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [51]:
!pip install -U bitsandbytes

In [53]:
# tokenizer = initialize_tokenizer(model_name)
# model = load_quantized_model(model_name)

In [ ]:
llms = HuggingFacePipeline(pipeline=pipeline)

In [ ]:

# pipeline = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     use_cache=True,
#     device_map="auto",
#     max_length=2048,
#     do_sample=True,
#     top_k=5,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
#     pad_token_id=tokenizer.pad_token_id,
# )

In [ ]:
# !wget https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_0.gguf?download=true

--2024-08-01 17:49:55--  https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_0.gguf?download=true
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.23, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/fe/17/fe17596731f84a0d03bece77489780bc7e068323c0aeca88b6393d3e9e65dd49/b525eb8dccd26c1b439dd0772b3b4db9ce3755e3046792c8e622683bd8f10b03?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27zephyr-7b-beta.Q4_0.gguf%3B+filename%3D%22zephyr-7b-beta.Q4_0.gguf%22%3B&Expires=1722793795&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjc5Mzc5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ZlLzE3L2ZlMTc1OTY3MzFmODRhMGQwM2JlY2U3NzQ4OTc4MGJjN2UwNjgzMjNjMGFlY2E4OGI2MzkzZDNlOWU2NWRkNDkvYjUyNWViOGRjY2QyNmMxYjQzOWRkMDc

In [56]:
from langchain.llms import LlamaCpp
llms = LlamaCpp(streaming=True,
                   model_path="/content/model/zephyr-7b-beta.Q4_0.gguf",
                   max_tokens = 500,
                   temperature=0.5,
                   top_p=1,
                   gpu_layers=0,
                   stream=True,
                   verbose=True,n_threads = int(os.cpu_count()/2),
                   n_ctx=4096)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/model/zephyr-7b-beta.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llam

In [57]:
from langchain.chains import RetrievalQA

In [58]:
qa = RetrievalQA.from_chain_type(
      llm=llms,
      chain_type="stuff",
      retriever = compression_retriever_reordered,
      return_source_documents = True
)

In [59]:
import os
os.listdir('/content')

['.config',
 '.ipynb_checkpoints',
 'Transformer.pdf',
 'model',
 'Retrieval-Augmented-Generation-for-NLP.pdf',
 'sample_data']

In [62]:
result = qa("Explain multi head attention?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2877.94 ms
llama_print_timings:      sample time =     103.85 ms /   166 runs   (    0.63 ms per token,  1598.51 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =  147798.93 ms /   166 runs   (  890.36 ms per token,     1.12 tokens per second)
llama_print_timings:       total time =  148109.89 ms /   166 tokens


In [64]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [65]:
# query ="what is encoders?"
# results = qa(query)
# print(results['result'])
#
# print(results["source_documents"])
result['result']

' Multi Head Attention is a technique used in the Transformer architecture of neural networks. It allows the model to jointly attend to information from different representation subspaces at different positions by performing multiple instances of self-attention with different learned linear projections. This helps to overcome the limitations of single attention head, such as averaging inhibiting the ability to attend to multiple positions simultaneously. Multi Head Attention consists of several attention layers running in parallel, each with its own set of query, key, and value projections. The outputs of these parallel attention layers are then concatenated and projected again, resulting in the final values. This technique has been found beneficial for a variety of tasks such as reading comprehension, abstractive summarization, textual entailment, and learning task-independent sentence representations.'

In [66]:
#


 Jon Snow is a character in George R.R. Martin's "A Song of Ice and Fire" series, which has been adapted into the popular HBO show "Game of Thrones." In the context provided, he is described as Bran Stark's bastard brother and is mentioned as moving closer to Bran during a scene in which Bran witnesses his father, King Robert Baratheon, sentencing Ned Stark to death for treason. Jon Snow has been portrayed by actor Kit Harington on the TV show "Game of Thrones."

In [67]:
results = qa("how the transformer architecture helps in building rag pipeline?")
print(results['result'])


Llama.generate: prefix-match hit

llama_print_timings:        load time =    2877.94 ms
llama_print_timings:      sample time =     121.07 ms /   195 runs   (    0.62 ms per token,  1610.64 tokens per second)
llama_print_timings: prompt eval time =  620031.23 ms /  1455 tokens (  426.14 ms per token,     2.35 tokens per second)
llama_print_timings:        eval time =  166545.38 ms /   194 runs   (  858.48 ms per token,     1.16 tokens per second)
llama_print_timings:       total time =  786947.63 ms /  1649 tokens


 The Transformer, proposed in this work, is a model architecture that replaces recurrence with attention mechanisms, allowing for significantly more parallelization and enabling it to draw global dependencies between input and output sequences. This approach, as demonstrated by the authors' experiments, results in a new state of the art in translation quality after being trained for only twelve hours on eight P100 GPUs. The RAG (Retrieval-augmented Generation) pipeline, proposed in this paper, utilizes the Transformer architecture to learn to retrieve relevant information for the task and generates responses using either a sequence or token-level decoder. By learning to retrieve relevant information, as evidenced by ablations where the retriever is frozen during training, RAG achieves more diverse results than BART without needing any diversity-promoting decoding. The authors also provide scripts for running experiments with RAG and an interactive demo of a RAG model.


Harry Potter is the main character in J.K. Rowling's series of novels and films about magic and wizards. He is an orphan who discovers that he has magical powers and goes to attend Hogwarts School of Witchcraft and Wizardry, where he makes friends and battles evil forces like Lord Voldemort.

In [68]:
#
print(results["source_documents"])
#


[_DocumentWithState(metadata={'page': 2, 'source': '/content/Transformer.pdf'}, page_content='Figure 1: The Transformer - model architecture.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-', state={'embedded_doc': [0.025233056396245956, -0.014825851656496525, -0.01302702073007822, -0.028497273102402687, 0.010001987218856812, -0.006481351796537638, -0.005541581194847822, -0.005433953832834959, 0.009927929379045963, 0.037419356405735016, 0.04243699461221695, 0.004794339183717966, 0.023254549130797386, -0.05233170464634895, -0.021803954616189003, 0.04367738217115402, -0.034350693225860596, -0.01462731

In [69]:
for source in  results["source_documents"]:
    print(source.metadata)

{'page': 2, 'source': '/content/Transformer.pdf'}
{'page': 1, 'source': '/content/Transformer.pdf'}
{'page': 7, 'source': '/content/Transformer.pdf'}
{'page': 9, 'source': '/content/Transformer.pdf'}
{'page': 5, 'source': '/content/Transformer.pdf'}
{'page': 9, 'source': '/content/Retrieval-Augmented-Generation-for-NLP.pdf'}
{'page': 5, 'source': '/content/Retrieval-Augmented-Generation-for-NLP.pdf'}
{'page': 3, 'source': '/content/Retrieval-Augmented-Generation-for-NLP.pdf'}
{'page': 6, 'source': '/content/Retrieval-Augmented-Generation-for-NLP.pdf'}
{'page': 16, 'source': '/content/Retrieval-Augmented-Generation-for-NLP.pdf'}
